In [98]:
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import math,cmath
from scipy.sparse import diags
from scipy.integrate import quad
from mpmath import *

ModuleNotFoundError: No module named 'mpmath'

In [94]:
Nmst = 60; #Number of lattice points
Pbst = 50;#Probe attachment site
lbd = 0.0 ;#Lambda strength ofAAH
irrb = (1 + np.sqrt(5))/2
t = 1.0; # hopping potential for sites
to = 3.0; # hopping potential for bath
sitegammaindx = [0, Nmst-1, Pbst-1]
sitegammastrn = [1.0, 1.0, 0.0]
siteindx = np.array(range(1, Nmst+1))
sitepotential = 2*lbd*np.cos(2*np.pi*irrb*(siteindx))
diagonals = [sitepotential,t*np.ones(Nmst-1), t*np.ones(Nmst-1)]
offset = [0,-1,1]
sys_Ham = diags(diagonals,offset,dtype='complex_').toarray()
mu_l = 1.4
mu_r = 1.8
mu_p = 1.2
beta_left = 1/10
beta_right = 1/10
beta_probe = 1/20
def bosonic_distribution(mu, energy, beta):#bosonic distribution function
    return 1/(np.exp(beta*(energy - mu)))
def selfenergy(gamma,site,energy):#self energy matrix
    mat =     mat = np.zeros((Nmst,Nmst),dtype = 'complex_')
    mat[site,site] =  ((gamma**2/(2*to**2))*(energy - cmath.sqrt(4*to**2-energy**2)*1j))
    return mat
def specden(gamma,energy):#spectral density matrix(-2Im(sigma))
    mat =  -2*((gamma**2/(2*to**2))*(energy - cmath.sqrt(4*to**2-energy**2)*1j)).imag
    return mat
def Green_func(energy):#green's function
    mat = energy*np.identity(Nmst) - sys_Ham - selfenergy(sitegammastrn[0],sitegammaindx[0],energy) - \
    selfenergy(sitegammastrn[1],sitegammaindx[1],energy) - selfenergy(sitegammastrn[2],sitegammaindx[2],energy)
    return (np.linalg.det(mat)/t)
def advan_gren(energy):
    mat = np.transpose(np.conjugate(Green_func(energy)))
    return mat
def transmissionprob(sitindx1, sitstrn1, sitindx2, sitstrn2, energy):
    spcdn1 = specden(sitstrn1, energy)
    retgre = Green_func(energy)
    spcdn2 = specden(sitstrn2, energy)
    advgre = advan_gren(energy)
    mat = (spcdn1*spcdn2)/(abs(retgre)**2)
    return mat

In [95]:
def current_val(energy):#integral equation
    first_trans = transmissionprob(sitegammaindx[2], sitegammastrn[2], sitegammaindx[0], sitegammaindx[0],energy)
    left_bath = bosonic_distribution(mu_l, energy, beta_left)
    probe = bosonic_distribution(mu_p, energy, beta_probe)
    right_bath = bosonic_distribution(mu_r, energy, beta_right)
    second_trans = transmissionprob(sitegammaindx[0], sitegammastrn[0], sitegammaindx[1], sitegammaindx[1],energy)
    return first_trans*(left_bath - probe) + second_trans*(right_bath - probe)
J, err  = quadosc(current_val, [-inf,inf])
point = np.linspace(-5,5, 1500)
current_val2 = np.vectorize(current_val)
I = current_val2(point)
plt.plot(point,I)
plt.show()
    

NameError: name 'quadsoc' is not defined